In [1]:
import pickle

In [2]:
from GeneralisedWick import *

In [3]:
fockTensor = Tensor("f", ['g'], ['g'])
h1Tensor = Tensor("h", ['g'], ['g'])
h2Tensor = Tensor("v", ['g', 'g'], ['g', 'g'])
t2Tensor = Tensor("{t_{2}}", ['p', 'p'], ['h', 'h'])

In [4]:
fockTensor.getAllDiagramsGeneral()
h2Tensor.getAllDiagramsGeneral()

In [5]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [6]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [7]:
def projectionManifold(excitationLevel):
    return Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

In [8]:
normalOrderedHamiltonian = sum(fockTensor.diagrams) + (1. / 2.) * sum(h2Tensor.diagrams)
BCHTransformedNormalOrderedHamiltonian = BCHSimilarityTransform(normalOrderedHamiltonian, 0.25 * t2Tensor, 4)

In [9]:
def pickleEquation(equation, name):
    with open(name + ".pkl", "wb") as f:
        p = pickle.Pickler(f)
        p.dump(equation)

In [10]:
def unpickleEquation(name):
    with open(name + ".pkl", "rb") as f:
        up = pickle.Unpickler(f)
        return up.load()

In [9]:
CCDEnergyEquation = evaluateWick(BCHTransformedNormalOrderedHamiltonian, True)

In [12]:
pickleEquation(CCDEnergyEquation, "CCDEnergyEquation")

In [13]:
UnpickledCCDEnergyEquation = unpickleEquation("CCDEnergyEquation")

In [18]:
print(*UnpickledCCDEnergyEquation.summandList[0].tensorList)

v_{hh}^{pp} {t_{2}}_{pp}^{hh}


In [19]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import pickle',
  'from GeneralisedWick import *',
  'fockTensor = Tensor("f", [\'g\'], [\'g\'])\nh1Tensor = Tensor("h", [\'g\'], [\'g\'])\nh2Tensor = Tensor("v", [\'g\', \'g\'], [\'g\', \'g\'])\nt2Tensor = Tensor("{t_{2}}", [\'p\', \'p\'], [\'h\', \'h\'])',
  'fockTensor.getAllDiagramsGeneral()\nh2Tensor.getAllDiagramsGeneral()',
  'def commutator(operator1, operator2):\n    return operator1 * operator2 + (-1) * operator2 * operator1',
  'def BCHSimilarityTransform(H, T, order):\n    result = H\n    for k in range(order):\n        nestedCommutator = H\n        for i in range(k + 1):\n            nestedCommutator = commutator(nestedCommutator, T)\n        result += (1 / factorial(k + 1)) * nestedCommutator\n   

In [10]:
def save(filename, *args):
    # Get global dictionary
    glob = globals()
    d = {}
    for v in args:
        # Copy over desired values
        d[v] = glob[v]
    with open(filename, 'wb') as f:
        # Put them in the file 
        pickle.dump(d, f)

def load(filename):
    # Get global dictionary
    glob = globals()
    with open(filename, 'rb') as f:
        for k, v in pickle.load(f).items():
            # Set each global variable to the value from the file
            glob[k] = v

In [37]:
def collectTensors(equation):
    return set([tensor for summand in equation.summandList for tensor in summand.tensorList])

In [38]:
def saveEquation(filename, equation):
    save(filename, equation, *collectTensors(equation))

In [11]:
save("testCCDSave.pkl", "CCDEnergyEquation", "fockTensor", "h2Tensor", "t2Tensor")

In [11]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'globals()',
  'import pickle',
  'from GeneralisedWick import *',
  'get_ipython().run_line_magic(\'load\', \'("CCDEnergyEquation.pkl")\')',
  "def save(filename, *args):\n    # Get global dictionary\n    glob = globals()\n    d = {}\n    for v in args:\n        # Copy over desired values\n        d[v] = glob[v]\n    with open(filename, 'wb') as f:\n        # Put them in the file \n        pickle.dump(d, f)\n\ndef load(filename):\n    # Get global dictionary\n    glob = globals()\n    with open(filename, 'rb') as f:\n        for k, v in pickle.load(f).items():\n            # Set each global variable to the value from the file\n            glob[k] = v",
  'load("CCDEnergyEquation.pkl")',
  "def save(filename, *a

In [10]:
load("testCCDSave.pkl")

In [13]:
print(CCDEnergyEquation)

0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{1}}


In [29]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
#mol.atom = 'Be 0 0 0'
#mol.basis = 'sto-3g'
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = '6-31g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

h2Tensor.array = eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)

fock = h1
for p in range(Norbs):
    for q in range(Norbs):
        fock[p,q] += sum([2 * h2Tensor.array[p,i,q,i] - h2Tensor.array[p,i,i,q] for i in range(Nocc)])
fockTensor.array = fock

fockTensor.assignDiagramArrays(vacuum)
h2Tensor.assignDiagramArrays(vacuum)
t2Tensor.getShape(vacuum)

In [30]:
contractTensorSum(CCDEnergyEquation)

0.0

In [31]:
import CC

In [32]:
vacuum

[1, 0, 0, 0]

In [33]:
residualTensor = Tensor("R", ['p', 'p'], ['h', 'h'])
residualTensor.array = h2Tensor.diagrams[12].array

In [34]:
h2Tensor.diagrams[12].array.shape

(3, 3, 1, 1)

In [35]:
CC.iterateDoublesAmplitudes(t2Tensor, residualTensor, fockTensor.array)

array([[[[-5.93124751e-02]],

        [[-5.10245496e-17]],

        [[-3.08991288e-02]]],


       [[[-1.55911290e-17]],

        [[-3.70716089e-02]],

        [[-2.80018313e-17]]],


       [[[-3.08991288e-02]],

        [[-3.37369716e-19]],

        [[-3.41821777e-02]]]])

In [36]:
contractTensorSum(CCDEnergyEquation)

-0.009291639221900173